In [ ]:
import numpy as np
import random
import math
import pandas as pd
import seaborn as sns
import scipy
from tqdm import tqdm
from Data.datasets import save_obj, load_obj

# import active learning data csv files

In [ ]:
df = pd.read_csv('Data/006.morph phase mapping.csv')
df.index = list(df['index'])
df = df.drop(['index'], axis = 1)
df_score = df['score']

df_pool = load_obj('8R homogeneous concentration statespace_new features_standardized (Pb2, morph, H2O and FAH constrained)')
df_pool_nonstd = load_obj('8R homogeneous concentration statespace_new features (Pb2, morph, H2O and FAH constrained)')

df = df_pool.filter(df.index, axis = 'index')
df = df.filter(['morph', 'Pb', 'FAH', 'H2O'], axis = 1)
df['score'] = df_score

df_nonstd = df_pool_nonstd.filter(df.index, axis = 'index')
df_nonstd = df_nonstd.filter(['morph', 'Pb', 'FAH', 'H2O'], axis = 1)
df_nonstd['score'] = df_score

# Convert AL dataset into binary
df_bin = df.copy()
df_bin = df_bin.drop(df_bin.index[df_bin['score']==1], axis = 0) # remove clear solution from the dataset

df_bin_nonstd = df_nonstd.copy()
df_bin_nonstd = df_bin_nonstd.drop(df_bin_nonstd.index[df_bin_nonstd['score']==1], axis = 0)

score_bin = []
for i in np.array(df_bin['score']):
    score_bin.append(1) if i == 3 else score_bin.append(0)
    
df_bin['score'] = score_bin # binary 0 (class 4) and 1 (class 3)
df_bin_nonstd['score'] = score_bin # binary 0 (class 4) and 1 (class 3)

# Check if there is a plane that can separate yellow phase and red phase

In [ ]:
df_fit = df_bin_nonstd.copy()

### Fit the whole dataset

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(solver='lbfgs')
LR.fit(np.array(df_fit.drop(['score'], axis = 1)),\
       np.array(df_fit.filter(['score'], axis = 1)).ravel())

print('accuracy: ', LR.score(np.array(df_fit.drop(['score'], axis = 1)), np.array(df_fit.filter(['score'], axis = 1)).ravel()))

In [ ]:
LR.coef_[0]

In [ ]:
LR.intercept_

### check cross validation for the regression model

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
accuracy_arr = []
coef_arr = []
intercept_arr = []

for train_index, test_index in tqdm(cv.split(df_fit.drop(['score'], axis = 1), df_fit.filter(['score'], axis = 1))):
    
    LR = LogisticRegression(solver='lbfgs')
    x_train = np.array(df_fit.drop(['score'], axis =1).iloc[train_index])
    y_train = np.array(df_fit.filter(['score'], axis = 1).iloc[train_index]).ravel()
    
    x_test = np.array(df_fit.drop(['score'], axis =1).iloc[test_index])
    y_test = np.array(df_fit.filter(['score'], axis = 1).iloc[test_index]).ravel()
    
    LR.fit(x_train, y_train)
    accuracy_arr.append(accuracy_score(y_test, LR.predict(x_test)))
    coef_arr.append(list(LR.coef_[0]))
    intercept_arr.append(list(LR.intercept_))

accuracy_arr = np.array(accuracy_arr)
coef_arr = np.array(coef_arr)
intercept_arr = np.array(intercept_arr)

In [ ]:
accuracy_arr.mean()

### Visualization of LR plane

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

xx, yy = np.meshgrid(range(int(round(df_bin['morph/Pb'].min()))-1, int(round(df_bin['morph/Pb'].max()))+1), \
                     range(int(round(df_bin['FAH'].min()))-1, int(round(df_bin['FAH'].max()))+2))
zz = (-LR.coef_[0][0] * xx - LR.coef_[0][1] * yy - LR.intercept_[0])/LR.coef_[0][2]

%matplotlib notebook

# plot the surface
ax = plt.figure(figsize = (10,7)).gca(projection='3d')
ax.plot_surface(xx, yy, zz, alpha=0.2, color = 'gray')

ax.scatter(df_bin['morph/Pb'][df_bin['score'] == 0],\
           df_bin['FAH'][df_bin['score'] == 0],\
           df_bin['H2O'][df_bin['score'] == 0],\
           c = 'red', s = 20, alpha = 0.8, linewidths = 0, label = 'Red phase')
ax.scatter(df_bin['morph/Pb'][df_bin['score'] == 1],\
           df_bin['FAH'][df_bin['score'] == 1],\
           df_bin['H2O'][df_bin['score'] == 1],\
           facecolors='none', edgecolors='blue', linewidths = 1.5, c = 'blue', s = 50, alpha = 0.2, label = 'Yellow phase')

ax.set_xlim(-2, 5)
ax.set_ylim(-2, 4)
ax.set_zlim(-5, 5)
ax.set_xlabel('morph/Pb')
ax.set_ylabel('FAH')
ax.set_zlabel('H2O')
    
plt.legend()
plt.savefig('Graphs_2/3D projection_morph_Pb_FAH, H2O_stand for 6 AL.png', format = "png", transparent=True)

In [ ]:
'log(morph)', 'log(Pb)'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D



%matplotlib notebook

# plot the surface

fig = plt.figure()
ax = fig.add_subplot()

ax.scatter(df_fit['log(morph)'][df_fit['score'] == 0],\
           df_fit['log(Pb)'][df_fit['score'] == 0],\
           c = 'red', s = 20, alpha = 0.8, linewidths = 0, label = 'Red phase')
ax.scatter(df_fit['log(morph)'][df_fit['score'] == 1],\
           df_fit['log(Pb)'][df_fit['score'] == 1],\
           facecolors='none', edgecolors='blue', linewidths = 1.5, c = 'blue', s = 50, alpha = 0.2, label = 'Yellow phase')

ax.set_xlabel('ln(morph)')
ax.set_ylabel('ln(Pb)')
plt.legend()

#### Validate logistic regression fitting and resulting coefficient and intercept

In [ ]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

y_lr_predict = sigmoid(np.dot(np.array(df_bin.drop(['score'], axis = 1)), LR.coef_[0]) + LR.intercept_).round()

In [ ]:
y_true = np.array(df_bin.filter(['score'], axis = 1)).ravel()

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_lr_predict)

#### Create animation of the 3D plot with decision plane

In [ ]:
import matplotlib.animation

# Hypothesis test for feature effect

In [ ]:
# Set the parameters for bootstrapping
sub_sample_frac = 1 # fraction of data for each bootrapping
sub_sample_numb = round(len(df_bin)*sub_sample_frac) # number of data for each bootstrapping
sample_num = 1000 # Times of bootstrapping will be performed


from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(solver='lbfgs')
coeff_arr = [] # coefficient for all features in each bootstrapping

for iteration in tqdm(range(sample_num)):
    idx = random.choices(list(df_bin.index), k = sub_sample_numb) # sample index (with replacement)
    df_train = df_bin.filter(idx, axis = 0) # index the samples
    
    # fit the LR with samples
    if True: #(0 in list(df_train['score'])) & (1 in list(df_train['score'])):
        LR.fit(np.array(df_train.drop(['score'], axis = 1)),\
               np.array(df_train.filter(['score'], axis = 1)).ravel())
        coeff_arr.append(list(LR.coef_[0]))
    else:
        coeff_arr.append([0]*(len(df_train.columns)-1)) # if there is only one class in the sample, set the coefficient = 0

coeff_arr = np.array(coeff_arr)

In [ ]:
# convert the coefficient numpy array to coefficient dataframe, and save it to an obj and a csv
coeff_df = pd.DataFrame(data = coeff_arr, columns = df_fit.columns[:-1])
save_obj(coeff_df, 'LG_coefficients(slopes)_dataframe_red phase')
coeff_df.to_csv('LG_coefficients(slopes)_dataframe_red phase.csv')

In [ ]:
for i in coeff_df.columns:
    print('slope for',i, 'is', coeff_df[i].mean(), "+-", coeff_df[i].std())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

for i in range(coeff_arr.shape[1]):
    fig = plt.figure(figsize = (6,6))
    plt.style.use('seaborn-whitegrid')
    ax = fig.add_subplot()
    ax.hist(coeff_arr[:,i], density=False, bins =15, color = 'g', alpha = 1, linewidth = 1)
    plt.xlabel('Slope of logistic regression')
    plt.ylabel('Count')
    plt.title(df_bin.columns[i])
    plt.legend()
    #plt.grid(True)
    plt.savefig('Graphs_2/LG slope_'+df_bin.columns[i]+'_5 AL + KS_red phase as 1.svg', format = "svg", transparent=True, dpi = 1000)

In [ ]:
# Test null hypothesis: feature coefficient = 0
feat_index = 3
feat_mean = np.mean(coeff_arr[:,feat_index])
feat_std = np.std(coeff_arr[:,feat_index])

sig_lvl = 0.01/(df_bin.shape[1]-1) # Bonferroni correction (multiple testing correction)
p = 1-(sig_lvl) # calculate the confidence value

z0 = scipy.stats.norm.ppf(p)
z = (feat_mean-0)/(feat_std/math.sqrt(sample_num))
print ('feature: ', df_bin.columns[feat_index])
print ('z0 is ', z0)
print ('z is ', z)